In [ ]:
import sys
dlc_dir = '/Users/annateruel/phd_code/'
if dlc_dir not in sys.path:
    sys.path.append(dlc_dir)


In [ ]:
import os
import pandas as pd
import dlc.analysis.time_roi as time_roi
from dlc.analysis.time_roi import ROIDrawer
from dlc.data import Concatenate

Concatenate your data from h5 and video files

In [ ]:
dir = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-097/control'
concatenator = Concatenate(dir)
h5_concat= concatenator.concatenate_h5()
video_concat = concatenator.concatenate_videos('.avi')

In [ ]:
root_dir = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img'

for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        for condition in ['control', 'anosmic']:
            condition_dir = os.path.join(subdir_path, condition)
            
            if os.path.exists(condition_dir):
                concatenator = Concatenate(condition_dir)
                h5_concat = concatenator.concatenate_h5()
                video_concat = concatenator.concatenate_videos('.avi')

1. Drawing ROIS with ROIdrawer tool

In [ ]:
video = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-097/anosmic/concatenated_files/AD23-097-s7-anosmic_concatenated.avi'
savedir = os.path.dirname(video)

roi_drawer = ROIDrawer(video_path=video,save_dir=savedir, num_rois=2)
r = roi_drawer.draw_rois()

2. Time in ROIs

In [ ]:
video_dir = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-097/anosmic/concatenated_files'
scorer = 'DLC_resnet50_sr-ca2+imgNov17shuffle1_250000'
bodypart = 'head'

In [ ]:
pd.read_hdf('/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-097/anosmic/concatenated_files/AD23-097-s5-anosmic_concatenated_roi.h5')

In [ ]:
roi_ellipse = {}
for file in os.listdir(video_dir):
    if file.endswith('roi.h5'):
        roi_file_path = os.path.join(video_dir, file)
        print(f"Processing ROI file: {file}")

        try:
            roi_data = pd.read_hdf(roi_file_path)
            roi_groups = roi_data.groupby(['index', 'shape-type'])
            ellipse, _ = time_roi.EllipseROI.extract_ellipses(roi_groups)
            key = file.replace('_roi.h5', '')
            roi_ellipse[key] = ellipse
            print(f"Extracted ellipses from {file}: {len(ellipse)}")
        except Exception as e:
            print(f"Error processing {file}: {e}")

In [ ]:
print("\nContents of roi_ellipse dictionary:")
for key, ellipses in roi_ellipse.items():
    print(f"Key: {key}, Number of ROIs: {len(ellipses)}")
    for i, ellipse in enumerate(ellipses):
        print(f"  ROI {i}: Center={ellipse.center}, Radii={ellipse.rad}")
        

In [ ]:
rois = roi_ellipse['AD23-097-s1-anosmic_concatenated']
time_in_roi = time_roi.TimeinRoi(fps=60)
file_path = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-097/anosmic/concatenated_files/AD23-097-s1-anosmic_concatenated.h5'

for roi in rois:
    time_in_roi.add_roi(roi)

tracking_data = time_in_roi.extract_tracking_data(file_path, scorer, bodypart)
time_in_rois = time_in_roi.time_in_rois(tracking_data)
print(time_in_rois)

In [ ]:
time_in_roi = time_roi.TimeinRoi(fps=60)
results_df = time_in_roi.time_in_rois_dir(directory=video_dir, 
                                          rois=roi_ellipse, 
                                          scorer=scorer, 
                                          body_part=bodypart, 
                                          file_endswith='concatenated.h5', 
                                          filename_replace = '.h5')
print(results_df)